In [ ]:
!pip install numpy pandas tensorflow scikit-learn

In [ ]:
!pip install git+https://github.com/genentech/gReLU.git

  Cloning https://github.com/genentech/gReLU.git to /tmp/pip-req-build-zxtgavir
  Running command git clone --filter=blob:none --quiet https://github.com/genentech/gReLU.git /tmp/pip-req-build-zxtgavir
  Resolved https://github.com/genentech/gReLU.git to commit 21a2fad912986d496df5fef3badc024bdcc00eae
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!python setup.py install

python3: can't open file '/content/setup.py': [Errno 2] No such file or directory


In [ ]:
import pandas as pd
import grelu.resources
import grelu.sequence.format
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm  # Import tqdm for progress bar
import numpy as np

In [ ]:
# Load your CSV file without headers, treating all lines as sequences
data = pd.read_csv('/content/chr_filtered_results.csv', header=None)

# Load the Borzoi model from gReLU
model = grelu.resources.load_model(
    project="borzoi",
    model_name="human_fold0"
)

input_len = model.data_params["train_seq_len"]
chrom = "chr10"
input_start = 106017251
input_end = input_start + input_len

input_intervals = pd.DataFrame({
    'chrom':[chrom], 'start':[input_start], 'end':[input_end], "strand":["+"],
})

input_intervals

wandb: Currently logged in as: anony-mouse-956256718815952248. Use `wandb login --relogin` to force relogin
wandb: Downloading large artifact human_fold0:latest, 711.00MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:7.2


,chrom,start,end,strand
0,chr10,106017251,106541539,+


In [ ]:
import grelu.sequence.format

input_seqs = grelu.sequence.format.convert_input_type(
    input_intervals,
    output_type="strings",
    genome="hg38"
)
input_seq = input_seqs[0]

len(input_seq)

524288

In [ ]:
output_intervals = model.input_intervals_to_output_intervals(input_intervals)
output_intervals

,chrom,start,end,strand
0,chr10,106181091,106377699,+


In [ ]:
output_start = output_intervals.start[0]
output_end = output_intervals.end[0]
output_len = output_end - output_start
print(output_len)

196608


In [ ]:
# Iterate through the array and create dictionaries
'''
for entry in first_column_array:
    chrom, input_start, ref, alt = entry.split(':')
    input_start = int(input_start)
    input_end = input_start + input_len  # Calculate the input_end

    # Create a dictionary and append it to the list
    dict_entry = {
        'chrom': chrom,
        'input_start': input_start,
        'input_end': input_end
    }
    dict_list.append(dict_entry)

# Example output
for item in dict_list[:5]:  # Print first 5 for preview
    print(item)

expected_length = model.data_params['train_seq_len']
'''

"\nfor entry in first_column_array:\n    chrom, input_start, ref, alt = entry.split(':')\n    input_start = int(input_start)\n    input_end = input_start + input_len  # Calculate the input_end\n\n    # Create a dictionary and append it to the list\n    dict_entry = {\n        'chrom': chrom,\n        'input_start': input_start,\n        'input_end': input_end\n    }\n    dict_list.append(dict_entry)\n\n# Example output\nfor item in dict_list[:5]:  # Print first 5 for preview\n    print(item)\n\nexpected_length = model.data_params['train_seq_len']\n"

In [ ]:
import grelu.visualize
%matplotlib inline

In [ ]:
tasks = pd.DataFrame(model.data_params['tasks'])
tasks.head(3)

,name,file,clip,clip_soft,scale,sum_stat,strand_pair,description,assay,sample
0,CNhs10608+,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.0,sum,1,CAGE:Clontech Human Universal Reference Total ...,CAGE,"Clontech Human Universal Reference Total RNA, ..."
1,CNhs10608-,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.0,sum,0,CAGE:Clontech Human Universal Reference Total ...,CAGE,"Clontech Human Universal Reference Total RNA, ..."
2,CNhs10610+,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.0,sum,3,CAGE:SABiosciences XpressRef Human Universal T...,CAGE,SABiosciences XpressRef Human Universal Total ...


In [ ]:
preds = model.predict_on_seqs(input_seqs, device=0)
preds.shape

OutOfMemoryError: CUDA out of memory. Tried to allocate 1024.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 203.06 MiB is free. Process 6799 has 14.55 GiB memory in use. Of the allocated memory 14.37 GiB is allocated by PyTorch, and 53.22 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:

cage_brain_tasks = tasks[(tasks.assay=="CAGE") & (tasks["sample"].str.contains("brain"))].head(2)
rna_brain_tasks = tasks[(tasks.assay=="RNA") & (tasks["sample"].str.contains("brain"))].head(2)

tasks_to_plot = cage_brain_tasks.index.tolist() + rna_brain_tasks.index.tolist()
task_names = tasks.description[tasks_to_plot].tolist() # Description of these tracks from the `tasks` dataframe

print(tasks_to_plot)
print(task_names)

fig = grelu.visualize.plot_tracks(
    preds[0, tasks_to_plot, :], # Outputs to plot
    start_pos=output_start, # Start coordinate for the x-axis label
    end_pos=output_end, # End coordinate for the x-axis label
    titles=task_names, # titles for each track
    figsize=(20, 6), # width, height
)

In [ ]:
'''
def pad_sequence(seq, length):
    return seq + 'A' * (length - len(seq)) if len(seq) < length else seq[:length]

# Add a progress bar for padding sequences
input_seqs = [pad_sequence(seq, expected_length) for seq in tqdm(sequences, desc="Padding sequences")]

# Use Borzoi to make predictions on the input sequences
print("Running Borzoi model predictions...")

# Add a progress bar for predictions
predictions = []
for seq in tqdm(input_seqs, desc="Predicting sequences"):
    pred = model.predict_on_seqs([seq], device="cpu")  # Predict one sequence at a time to show progress
    predictions.append(pred)

output_intervals = model.input_intervals_to_output_intervals(input_intervals)
output_intervals

output_start = output_intervals.start[0]
output_end = output_intervals.end[0]
output_len = output_end - output_start
print(output_len)

# Convert predictions list to numpy array
predictions = np.vstack(predictions)

# Borzoi outputs predictions (e.g., probability tracks); use these as features
# Flatten the predictions for Random Forest input
predictions_flat = predictions.reshape(predictions.shape[0], -1)
'''